In [1]:
# some standard packages
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import os
from random import shuffle

# modelling packages
# import tensorflow as tf
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
# from tensorflow.keras.optimizers import RMSprop
# from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau 
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
# from tensorflow.keras.models import model_from_json
# from tensorflow.keras.models import load_model

# Importing tf tools
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Model, Sequential

# Model Evaluation
from sklearn.metrics import classification_report, confusion_matrix


Utilize the cell below only if using all of the Data Loader file instead of using the Keras Image Generator.

In [2]:
# Open the data files
# X = pickle.load(open('X.pickle', 'rb'))
# y = pickle.load(open('y.pickle', 'rb'))

In [3]:
categories = ['class_0', 'class_1', 'class_2', 'class_3']

In [4]:
# # Creating a nested for-loop that will move 20% of the images for each class into a test set
# for each_category in categories: 
    
#     # Make the directory
#     #os.mkdir(path + f'test/{each_category}')
    
#     # Getting a list of the images
#     list_images = [file for file in os.listdir(f'data/BC-images-clean/{each_category}') if file.endswith('.jpg')]
    
#     # Randomly shuffling the order 
#     shuffle(list_images)
    
#     # Getting the names of the first 20% of images
#     num_test_images = int(len(list_images)*0.2)
#     for_testset = list_images[ : num_test_images]
    
#     # Looping through each image in this list 'for_testset'
#     for each_image in for_testset:
        
#         # Renaming the file path to move those image to test set
#         os.rename(f'data/BC-images-clean/{each_category}/{each_image}', 
#                   f'data/BC-images-clean/test/{each_category}/{each_image}')
        
#         # Sanity check
#         print(f'{each_image} moved to "{each_category}" in test.')
    
#     # Another check
#     print(f'{each_category} completed.')

In [5]:
# Use the Keras ImageDataGenerator for memory efficiency and preprocessing ease
# This process replaces the method of obtaining our data via DataLoader.ipynb
train_datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2,
)

In [6]:
test_datagen = ImageDataGenerator(rescale = 1./255)

In [7]:
# train_img_dir = 'Data/train/'
# test_img_dir = 'Data/test/'

In [8]:
batch_size = 32

In [9]:
# # Open the data files
# train = pickle.load(open('train_labels.pickle', 'rb'))
# test = pickle.load(open('test_labels.pickle', 'rb'))

In [10]:
# train.head()

In [11]:
# train['Label'].value_counts()

In [12]:
# train.shape

In [13]:
train_generator = train_datagen.flow_from_directory(
                                                    'data/BC-images-clean/train',
                                                    target_size=(32, 32),
                                                    color_mode='rgb',
                                                    batch_size=batch_size,
                                                    class_mode='categorical',
                                                    shuffle=True,
                                                    subset='training')

Found 5387 images belonging to 4 classes.


In [14]:
validation_generator = train_datagen.flow_from_directory(
                                                        'data/BC-images-clean/train',
                                                        target_size=(32, 32),
                                                        color_mode='rgb',
                                                        batch_size=batch_size,
                                                        class_mode='categorical',
                                                        shuffle=False,
                                                        subset='validation'
                                                        )

Found 1344 images belonging to 4 classes.


In [15]:
test_generator = test_datagen.flow_from_directory(
                                                  'data/BC-images-clean/test',
                                                  target_size=(32, 32),
                                                  color_mode='rgb',
                                                  batch_size=batch_size,
                                                  class_mode='categorical',
                                                  shuffle=False)

Found 1680 images belonging to 4 classes.


In [16]:
# Saving the number of stepsizes for the training, validation and test sets 
train_stepsize = train_generator.samples//train_generator.batch_size 

valid_stepsize = validation_generator.samples//validation_generator.batch_size 

test_stepsize = test_generator.samples//test_generator.batch_size 

# Sanity check 
print(f'Training step size = {train_stepsize} \nValidation step size = {valid_stepsize} \nTest step size = {test_stepsize}')

Training step size = 168 
Validation step size = 42 
Test step size = 52


In [17]:
# 1.  Import the pretrained VGG16 network, do not include the top layers
pretrained = VGG16(weights='imagenet', include_top=False, pooling='max', input_shape=(32, 32, 3))

# 2.  Setting all layers to not trainable so weights wont be tweaked
for layer in pretrained.layers:
    layer.trainable=False
    
# Display VGG16 architecture
pretrained.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0     

In [18]:

# Instantiate the NN model 
weeds_model = Sequential()

# Add the pretrained layers 
weeds_model.add(pretrained)

# Add fully-connected dense layers -- plus a dropout layer to help prevent overfitting
weeds_model.add(Dense(256, activation='relu'))
weeds_model.add(Dropout(0.5))
weeds_model.add(Dense(512, activation='relu'))

# Adding our activation 
weeds_model.add(Dense(4, activation='softmax'))

In [19]:
# Initiate early stop based on validation accuracy
ES = EarlyStopping(monitor='val_acc', patience=5, mode='auto', min_delta=0.0001, verbose=1)

In [20]:
# Istantiating Adam optimizer with a learning rate of 0.0001 and saving to variable 'optim'
optim = Adam(lr=0.0001)

# Compiling the CNN model 
weeds_model.compile(optimizer=optim, loss='categorical_crossentropy', metrics=['acc'])

# Summary 
weeds_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 512)               14714688  
_________________________________________________________________
dense (Dense)                (None, 256)               131328    
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               131584    
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 2052      
Total params: 14,979,652
Trainable params: 264,964
Non-trainable params: 14,714,688
_________________________________________________________________


In [21]:
# Fitting the model to the training data
history = weeds_model.fit_generator(generator=train_generator,
                                steps_per_epoch=train_stepsize,
                                epochs=70,
                                validation_data=validation_generator,
                                validation_steps=valid_stepsize,
                                callbacks=[ES])

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/70
168/168 [==============================] - 105s 626ms/step - loss: 1.1829 - acc: 0.5007 - val_loss: 1.0527 - val_acc: 0.5543
Epoch 2/70
168/168 [==============================] - 102s 608ms/step - loss: 1.0573 - acc: 0.5501 - val_loss: 0.9549 - val_acc: 0.5960
Epoch 3/70
168/168 [==============================] - 104s 620ms/step - loss: 0.9883 - acc: 0.5802 - val_loss: 0.9016 - val_acc: 0.6116
Epoch 4/70
168/168 [==============================] - 109s 650ms/step - loss: 0.9579 - acc: 0.5987 - val_loss: 0.8692 - val_acc: 0.6481
Epoch 5/70
168/168 [==============================] - 109s 652ms/step - loss: 0.9241 - acc: 0.5998 - val_loss: 0.8562 - val_acc: 0.6272
Epoch 6/70
168/168 [==============================] - 110s 654ms/step - loss: 0.9168 - acc: 0.6181 - val_loss: 0.8182 - val_acc: 0.6629
Epoch 7/70
168/168 [==============================] - 110s 655ms/step - loss: 0.8943 - acc: 0.6258 - val_los

In [22]:
# Getting bestmodel's predictions (as probabilities) on the test set 
test_probas = weeds_model.predict_generator(test_generator, steps=test_stepsize)

# Setting the model's class prediction as the class that received the highest probability for each image
test_predictions = test_probas.argmax(axis=1)

Instructions for updating:
Please use Model.predict, which supports generators.


In [31]:
len(test_probas)

1664

In [30]:
# Getting the true class labels for the test set
test_true = test_generator.classes

# Sanity check 
len(test_true)

1680

In [29]:
len(test_predictions)

1664

In [25]:
# Displaying the classification report for the test set
print('Classification Report\n \n', classification_report(test_true, test_predictions, target_names=categories))

ValueError: Found input variables with inconsistent numbers of samples: [1680, 1664]

In [26]:
# Get a confusion matrix 
test_matrix = pd.DataFrame(confusion_matrix(test_true, test_predictions), 
                           columns=['Predicted ' + cat_name for cat_name in categories], 
                           index=['True ' + cat_name for cat_name in categories])

# Plotting as a heatmap 
plt.figure()
sns.heatmap(test_matrix, cmap='Blues', annot=True, fmt='g')
plt.title('Normalized Confusion Matrix: Test Data')
plt.show()

ValueError: Found input variables with inconsistent numbers of samples: [1680, 1664]

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, stratify = y) 

In [ ]:
# # Building the Model
# model = Sequential()

# # # 3 convolutional layers
#model.add(Conv2D(28, (3,3), input_shape = (28,28,3)))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2,2)))

# model.add(Conv2D(64, (3, 3)))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))

# model.add(Conv2D(64, (3,3)))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2,2)))
# model.add(Dropout(0.25))

# # # 2 hidden layers
# model.add(Flatten())
# model.add(Dense(128))
# model.add(Activation('relu'))

# model.add(Dense(128))
# model.add(Activation('relu'))

# # # The output layer with 9 neurons for 9 classes
# model.add(Dense(9))
# model.add(Activation('softmax'))

# # # Compiling the model using some basic parameters
# model.compile(loss='sparse_categorical_crossentropy',
#               optimizer='adam',
#               metrics=['accuracy'])

In [ ]:
#train_generator.filenames

In [ ]:
# # Train the model
# history = model.fit_generator(generator=train_generator,
#                     steps_per_epoch=(11209) // batch_size,
#                     epochs=50, 
#                     validation_data=validation_generator,
#                     validation_steps=(2798) // batch_size,
#                     callbacks=[
#                         EarlyStopping(patience=3, restore_best_weights=True),
#                         ReduceLROnPlateau(patience=2)],
#                     verbose=1)